**1 Подгрузим данные и необходимые библиотеки**

In [1]:
# библиотека для скачивания данных с гугл.диска
!pip install --upgrade --no-cache-dir gdown

In [2]:
# библиотека для векторизации текста, обновим до актуальной версии
!pip install -U gensim

     |████████████████████████████████| 24.1 MB 1.3 MB/s 
  Attempting uninstall: gensim
    Found existing installation: gensim 3.6.0
    Uninstalling gensim-3.6.0:
      Successfully uninstalled gensim-3.6.0


In [3]:
import gdown
import pandas as pd
import numpy as np
import random
import string
from tqdm import tqdm_notebook as tqdm
import re
from scipy import spatial
import heapq
import timeit
from ast import literal_eval

import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

from gensim.models import Word2Vec

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.


In [4]:
pd.set_option('display.max_columns', None)

Файлы были взяты из этой задачки: kaggle.com/datasets/rounakbanik/the-movies-dataset

In [5]:
!gdown 1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
!gdown 14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
!gdown 1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC

Downloading...
From: https://drive.google.com/uc?id=1Co7iQWJhJdqNFTlChd-xT1b88VAzuX4f
To: /content/movies_metadata.csv
100% 34.4M/34.4M [00:00<00:00, 107MB/s]
Downloading...
From: https://drive.google.com/uc?id=14OAYTUIw9BoCCSTFIiYzgPq-1XyP0dhQ
To: /content/ratings.csv
100% 710M/710M [00:07<00:00, 100MB/s]
Downloading...
From: https://drive.google.com/uc?id=1hrC11y_r0veFdZqkwH1MSUxSv_CwJtyC
To: /content/credits.csv
100% 190M/190M [00:01<00:00, 102MB/s]


**2 Проведем первичную обработку данных**

В первом датасете, которым мы воспользуемся, содержится вся основная информация о фильмах, например: бюджет, жанровая принадлежность, язык оригинала, название, описание, дата создания и тд.

In [6]:
movies_metadata = pd.read_csv('movies_metadata.csv')

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [7]:
movies_metadata.head(2)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,popularity,poster_path,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",21.946943,/rhIRbceoE9lR4veEXuwCC2wARtG.jpg,"[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,17.015539,/vzmL6fP7aPKNKPRTFnZmiUfciyV.jpg,"[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0


In [8]:
# удалим ошибочные значения
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '1997-08-20'].index, inplace=True)

In [9]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2012-09-29'].index, inplace=True)

In [10]:
movies_metadata.drop(movies_metadata[movies_metadata['id'] == '2014-01-01'].index, inplace=True)

В этом наборе мы сосредоточимся на текстовых данных и  будем работать только со стобцами original_title (название фильма) и overview (официальное описание фильма)

In [11]:
# оставим только колонки, которые мы собираемся использовать
movies_metadata = movies_metadata[['id', 'title', 'overview', 'genres', 'production_companies']]

In [12]:
# заполняем пустые значения
movies_metadata = movies_metadata.fillna('')

In [13]:
# удалим дубли, потому что они есть и очень мешаются
movies_metadata = movies_metadata.drop_duplicates().reset_index(drop=True)

In [14]:
movies_metadata = movies_metadata.rename(columns={'id':'movieId'})

In [15]:
# меняем тип данных колонки, чтобы проще было джойниться с другой таблицей
movies_metadata['movieId'] = movies_metadata['movieId'].astype(int)

Во втором наборе содержится информация о съемочной команде, нас будет интересовать имя режисера

In [16]:
credits = pd.read_csv('credits.csv')[['crew', 'id']]

In [17]:
credits = credits.rename(columns={'id':'movieId'})

In [18]:
credits = credits.drop_duplicates().reset_index(drop=True)

In [19]:
credits.head(5)

,crew,movieId
0,"[{'credit_id': '52fe4284c3a36847f8024f49', 'de...",862
1,"[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de...",8844
2,"[{'credit_id': '52fe466a9251416c75077a89', 'de...",15602
3,"[{'credit_id': '52fe44779251416c91011acb', 'de...",31357
4,"[{'credit_id': '52fe44959251416c75039ed7', 'de...",11862


Теперь поработаем с еще одним набором - пользовательскими оценками фильмов

In [20]:
ratings = pd.read_csv('ratings.csv')

# выберем только достаточно высоко оцененные фильмы
ratings = ratings[ratings['rating'] > 3]

После подгрузки всех необходимых данных найдем их пересечение по айди фильмов

Дело в том, что не для всех фильмов есть данные о просмотрах и оценках, а для некоторых оценок нет данных о самом фильме

In [21]:
common_ids = set(movies_metadata.movieId).intersection(set(ratings.movieId))

ratings = ratings[ratings.movieId.isin(common_ids)].reset_index(drop=True)
movies_metadata = movies_metadata[movies_metadata.movieId.isin(common_ids)].reset_index(drop=True)

In [22]:
movies_metadata.shape[0]

6835

In [23]:
movies_metadata.groupby('movieId').count().sort_values(by='genres', ascending=False).head(4)

,title,overview,genres,production_companies
movieId,,,,
2,1,1,1,1
65300,1,1,1,1
65216,1,1,1,1
65188,1,1,1,1


In [24]:
credits = credits[credits.movieId.isin(common_ids)].reset_index(drop=True)

In [25]:
# здесь мы видим, что размерность не совпадает - есть подозрительные записи
credits.shape[0]

6837

In [26]:
# ищем задвоенные, но не идентичные записи
credits.groupby('movieId').count().sort_values(by='crew', ascending=False).head(4)

,crew
movieId,
159849,2
4912,2
64650,1
65216,1


In [27]:
credits[credits['movieId'] == 159849]

,crew,movieId
3181,"[{'credit_id': '52fe4c229251416c910f10d5', 'de...",159849
6056,"[{'credit_id': '52fe4c229251416c910f10d5', 'de...",159849


In [28]:
credits.drop(credits[credits.index == 3181].index, inplace=True)

In [29]:
credits[credits['movieId'] == 4912]

,crew,movieId
1525,"[{'credit_id': '52fe43e2c3a36847f80760b5', 'de...",4912
5856,"[{'credit_id': '52fe43e2c3a36847f80760a9', 'de...",4912


In [30]:
credits.drop(credits[credits.index == 5856].index, inplace=True)

In [31]:
# убеждаемся, что получили нужный размер
credits.shape[0]

6835

In [32]:
# соединяем с основным набором
movies_metadata = pd.merge(movies_metadata, credits, on='movieId')

In [33]:
movies_metadata.head(4) 

,movieId,title,overview,genres,production_companies,crew
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...","[{'id': 16, 'name': 'Animation'}, {'id': 35, '...","[{'name': 'Pixar Animation Studios', 'id': 3}]","[{'credit_id': '52fe4284c3a36847f8024f49', 'de..."
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...","[{'name': 'TriStar Pictures', 'id': 559}, {'na...","[{'credit_id': '52fe44bfc3a36847f80a7cd1', 'de..."
2,949,Heat,"Obsessive master thief, Neil McCauley leads a ...","[{'id': 28, 'name': 'Action'}, {'id': 80, 'nam...","[{'name': 'Regency Enterprises', 'id': 508}, {...","[{'credit_id': '52fe4292c3a36847f802916d', 'de..."
3,710,GoldenEye,James Bond must unmask the mysterious head of ...,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...","[{'name': 'United Artists', 'id': 60}, {'name'...","[{'credit_id': '52fe426ec3a36847f801e14b', 'de..."


Cтолбцы genres, production_companies и crew имеют сложную структуру, но по факту их тип - строка, надо это исправить, чтобы достать из них необходимую нам информацию

In [34]:
type(movies_metadata['genres'][0])

str

In [35]:
# literal_eval позволяет восстановить исходную структуру признака из строки
movies_metadata['crew'] = movies_metadata['crew'].apply(literal_eval)
movies_metadata['genres'] = movies_metadata['genres'].apply(literal_eval)
movies_metadata['production_companies'] = movies_metadata['production_companies'].apply(literal_eval)

In [36]:
# получилось
type(movies_metadata['genres'][0])

list

Напишем и применим функции, достающие имена режисеров, названия студий и жанры фильмов

In [37]:
def get_director(x):
    for i in x:
        if i['job'] == 'Director':
            return i['name']
    return np.nan

In [38]:
def get_genres_company(x):
    for i in x:
        return i['name']
    return np.nan

In [39]:
movies_metadata['director'] = movies_metadata['crew'].apply(get_director)
movies_metadata['genres'] = movies_metadata['genres'].apply(get_genres_company)
movies_metadata['production_company'] = movies_metadata['production_companies'].apply(get_genres_company)

In [40]:
movies_metadata = movies_metadata[['movieId', 'title', 'overview', 'director', 'genres', 'production_company']]

In [41]:
movies_metadata.head(4)

,movieId,title,overview,director,genres,production_company
0,862,Toy Story,"Led by Woody, Andy's toys live happily in his ...",John Lasseter,Animation,Pixar Animation Studios
1,8844,Jumanji,When siblings Judy and Peter discover an encha...,Joe Johnston,Adventure,TriStar Pictures
2,949,Heat,"Obsessive master thief, Neil McCauley leads a ...",Michael Mann,Action,Regency Enterprises
3,710,GoldenEye,James Bond must unmask the mysterious head of ...,Martin Campbell,Adventure,United Artists


**3 С помощью обработки текста получим матрицу эмбедингов для каждого фильма**

Нам нужно перевести текстовые данные из названия и описания фильмов в формат, которым может оперировать компьютер - вектора эмбедингов. Перед векторизацией нужно предобработать данные и токенизировать их

После этого мы сможем выполнять операции над этими данными, например сравнивать между собой и находить расстояние между двумя фильмами

In [42]:
def preprocessing(data, lemmatizer = WordNetLemmatizer(), 
                  stop_words = stopwords.words('english')):
    
    '''
       на вход: 
       - data - датафрейм с описанием и названием фильмов
       - lemmatizer - класс для лемматизации текстов
       - stop_words - стоп-слова
      
       на выходе: 
       - список с предобработанными текстовыми данными
    '''
    
    texts = data.values.tolist()
    new_text = []

    for i in tqdm(range(len(texts))):
        text = texts[i]

        # приводим к нижнему регистру и дропаем стоп-слова
        punct_free = re.sub('[^a-zA-Z]', ' ', str(text).lower())
        punct_free = re.sub(r'\s+', ' ', punct_free)
        stop_free = " ".join([i for i in punct_free.split() if i not in stop_words])
        lemma_text = ' '.join([lemmatizer.lemmatize(word) for word in stop_free.split(' ')])
        new_text.append(lemma_text)
    
    return new_text

In [43]:
nltk.download("stopwords")

text_overview = preprocessing(movies_metadata["overview"])
text_title = preprocessing(movies_metadata["title"])
text_dir = preprocessing(movies_metadata["director"])
text_genres = preprocessing(movies_metadata["genres"])
text_company = preprocessing(movies_metadata["production_company"])

movies_metadata["overview"] = text_overview
movies_metadata["title"] = text_title 
movies_metadata["director"] = text_dir
movies_metadata["genres"] = text_genres
movies_metadata["production_company"] = text_company

# закидываем предобработанные описания и названия фильмов в общий список
text_prepr = text_overview + text_title + text_dir + text_genres + text_company

# и разбиваем на отдельные слова
text_prepr = [i.split() for i in text_prepr]
text_overview = [i.split() for i in text_overview]
text_title = [i.split() for i in text_title]
text_dir = [i.split() for i in text_dir]
text_genres = [i.split() for i in text_genres]
text_company = [i.split() for i in text_company]

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:17: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`


  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

  0%|          | 0/6835 [00:00<?, ?it/s]

In [44]:
# с помощью word2vec получаем векторизованный словарь
model = Word2Vec(text_prepr, 
                 min_count=2)

model.build_vocab(text_prepr)
words = model.wv.index_to_key
vocab_size = len(words)

print("Vocab size =", vocab_size)

Vocab size = 16575


После обработки всех текстовых данных перейдем к созданию матрицы эмбедингов для каждого фильма

In [45]:
def embedding_matrix(text, model):
    
    """
       На вход: словарь tokenizer.word_index 
       модель word2vec
        
       На выходе: матрица эмбедингов текста
    """
    
    embedding_matrix = np.zeros((id_film, 100))

    for sentence in text:
        for word in sentence:
            if word in model:
                embedding_matrix[text.index(sentence)] = model[word]

    return embedding_matrix

In [46]:
id_film = movies_metadata['movieId'].nunique()

# получим матрицы с эмбедингами для всех текстовых признаков
matrix_emb_over = embedding_matrix(text_overview, model.wv)
matrix_emb_title = embedding_matrix(text_title, model.wv)
matrix_emb_dir = embedding_matrix(text_dir, model.wv)
matrix_emb_genres = embedding_matrix(text_genres, model.wv)
matrix_emb_company = embedding_matrix(text_company, model.wv)

# и соединим их
matrix_emb = pd.concat([movies_metadata[['movieId']],
                        pd.DataFrame(matrix_emb_over),
                        pd.DataFrame(matrix_emb_title),
                        pd.DataFrame(matrix_emb_dir),
                        pd.DataFrame(matrix_emb_genres),
                        pd.DataFrame(matrix_emb_company)], axis=1)

In [47]:
matrix_emb.shape

(6835, 501)

In [48]:
matrix_emb.head(4)

,movieId,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99
0,862,0.461102,1.601728,0.938574,0.014139,-0.400293,-1.291074,-0.630294,1.220047,-0.255775,0.240805,-0.636164,-0.473432,-0.105059,-0.264048,0.087944,-0.144323,1.484419,-0.702344,0.036244,-1.716652,0.737569,1.110034,0.405295,-0.827932,-0.476178,0.372664,-1.783307,-0.750605,0.101526,0.342530,0.331588,0.517943,0.093523,-1.095678,-0.224398,2.018267,-0.143577,-0.797664,-0.415825,-2.169886,0.002883,-0.804186,0.375250,0.524413,-0.408893,-0.990319,-0.991435,-0.740315,-0.142567,0.191888,-0.180392,-0.909806,-0.709403,-0.869824,0.002642,-0.467789,-0.530700,-0.355868,-1.387921,0.325276,-0.102863,0.048225,-0.108708,1.087238,0.183726,0.813552,0.487273,-0.566945,-1.709100,0.289208,-2.267149,-0.638439,0.694896,-0.814048,0.896174,-0.033890,-0.209736,-0.255920,0.278499,0.391465,-0.212634,0.970731,-0.689595,1.394459,-0.399688,0.062578,0.731776,1.760576,0.819619,-0.234394,0.711859,-0.924834,0.773414,-0.407758,0.929959,-0.819920,-0.876096,0.205464,-0.496819,0.602284,-0.086678,0.764726,0.669147,0.008479,-0.138298,-1.348738,0.037235,0.994685,-0.183562,-0.348982,-0.203580,-1.105616,-0.045362,0.038520,0.500088,-0.444972,0.370830,-1.315823,0.483213,-0.983828,0.701897,0.593233,0.559801,-0.233016,-0.291137,0.484063,-0.987769,-0.257556,-0.524178,0.116143,0.298180,0.378721,-0.020354,-0.352769,-0.191275,1.003394,-0.150055,-0.690034,-0.631300,-1.195772,0.012169,-0.462631,-0.074163,0.035151,0.251307,-0.484860,-0.691344,-0.011245,0.197088,0.521380,0.240776,-0.374291,-0.218294,-0.279963,-0.388360,0.401660,0.266040,0.184976,-0.363206,-0.062063,-0.087176,0.258373,-0.324488,-0.057070,-0.518375,0.435348,-0.161781,0.024450,-0.826977,0.669517,-0.819680,0.360048,0.826736,-0.471962,0.728822,0.269378,0.028264,-0.183690,-0.722954,0.263830,-0.539026,0.206017,-0.383693,1.152590,-0.025956,-0.027107,-0.025987,0.970480,0.891376,0.264351,0.802313,-0.174218,0.174670,0.014188,0.814047,0.109398,0.231604,-0.471187,0.138580,0.193251,0.004644,0.014041,0.025314,0.007198,-0.002576,-0.043967,-0.006282,0.030919,0.000550,-0.013752,-0.012478,-0.033113,-0.010367,-0.007649,0.022780,-0.013750,0.006515,-0.033998,0.002615,-0.037361,0.014158,0.014622,0.018630,-0.011887,-0.009136,0.014144,-0.020465,-0.015504,-0.012839,0.002047,0.002747,0.015038,0.011416,-0.002141,-0.012940,0.032084,-0.013679,-0.015952,-0.014176,-0.026299,0.008891,-0.002551,-0.004856,-0.002091,0.013188,-0.004370,-0.014290,0.008634,0.015581,0.011401,0.007258,-0.012944,-0.004799,-0.012001,-0.008175,0.014583,0.002811,0.003991,-0.006089,0.001064,0.001375,0.011137,-0.011223,-0.008024,-0.006771,0.012540,-

**4 Напишем аналог функции train_test_split для пользователя и его предпочтений**


Подаем на вход функции юзера и получаем два списка фильмов: на тренировочном будем учить, а фильмы из тестового набора искать в предсказаниях

In [49]:
def train_test_split_films(dataframe, user_list, size=0.7):

  '''
     на вход подается:
     - dataframe - название датафрейма с рейтингами
     - user_list - список пользователей
     - size - пропорция разделения

     на выходе для всех пользователей получаем один словарь, где
     ключ - id зрителя
     список 1 - айдишники train фильмов
     список 2 - оценки train фильмов
     список 3 - айдишники test фильмов
     список 4 - оценки test фильмов
  '''

  user_tts_data_dict = {}

  # итерируемся по всем пользователям
  for user in user_list:

    # выбираю данные по нужному юзеру и перемешиваю
    temp_df = dataframe[dataframe['userId'] == user] \
              .sample(dataframe[dataframe['userId'] == user].shape[0]) \
              [['movieId', 'rating']]

    # отфильтровываем пользователей с количеством просмотров < 10
    if temp_df.shape[0] < 10:
      continue

    # вычисляю длину тренировочного набора для нужного юзера
    shape = int(temp_df.shape[0]*size)

    # сразу записываем в словарь четыре вектора
    user_tts_data_dict[user] = [temp_df.iloc[:shape]['movieId'].tolist(),
                                temp_df.iloc[:shape]['rating'].tolist(),
                                temp_df.iloc[shape:]['movieId'].tolist(),
                                temp_df.iloc[shape:]['rating'].tolist()]

  return user_tts_data_dict

In [50]:
# пользователей слишком много, мы (в силу отсутствия мощностей) не сможем 
# построить предсказания по всем, но можно выбрать сэмпл
len(ratings['userId'].unique())

258025

In [51]:
# вторым параметром на вход можно подать любой список id пользователей
splitted_data = train_test_split_films(ratings, ratings['userId'].unique()[:1000], 0.7)

Мы получили словарь, в котором для каждого пользователя, который посмотрел как минимум 10 фильмов, есть тренировочный набор фильмов и оценки к ним + тестовый набор фильмов и оценки к ним

**5 Для каждого пользователя по train-фильмам находим средний вектор предпочтений**

In [52]:
def get_avg_weightened_vec(dict_name, movies_emb_df):

  '''
     на вход подается:
     - dict_name - название словаря с данными после сплита 
     - movies_emb_df - название эмбединг-датасета с фильмами

     далее мы находим все эмбединг-вектора для train фильмов, домножаем
     каждый на оценку от зрителя и находим средний вектор для каждого зрителя

     на выходе:
     - словарь с ключами - айди пользователей и значениями - средним взвешенным 
     вектором по просмотренным фильмам в train-наборе по каждому пользователю
  '''

  user_vec_data_dict = {}

  # итерируемся по всем пользователям
  for key in dict_name.keys():

    # находим все вектора фильмов из train-выборки в эмбединг-матрице
    # и джойним с их рейтингами от выбранного пользователя
    temp_df = movies_emb_df \
              .loc[movies_emb_df['movieId'].isin(dict_name[key][0])] \
              .merge(pd.DataFrame(dict_name[key][0], dict_name[key][1], 
                                  columns=['movieId'])
                                  .reset_index()
                                  .rename(columns={'index':'rating'}), on='movieId')

    # домножаем эмбединг-вектора на веса-оценки и вычисляем средний вектор
    avg_weightened_vec = list(temp_df \
                              .drop(['movieId'], axis=1)
                              .mul(temp_df['rating'], axis=0)
                              .drop(['rating'], axis=1)
                              .sum()
                              .mul(1 / temp_df['rating'].sum(), axis=0))
    
    # все вектора записываем в словарь под ключ-айди пользователя
    user_vec_data_dict[key] = avg_weightened_vec

  return user_vec_data_dict

In [53]:
# получим матрицу пользователей
res_vec_dict = get_avg_weightened_vec(splitted_data, matrix_emb)

In [54]:
# преобразуем к табличному виду
res_vec_df = pd.DataFrame.from_dict(res_vec_dict, orient='index').reset_index().rename(columns={'index':'Id'})

In [55]:
res_vec_df.head(4)

,Id,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270,271,272,273,274,275,276,277,278,279,280,281,282,283,284,285,286,287,288,289,290,291,292,293,294,295,296,297,298,299,300,301,302,303,304,305,306,307,308,309,310,311,312,313,314,315,316,317,318,319,320,321,322,323,324,325,326,327,328,329,330,331,332,333,334,335,336,337,338,339,340,341,342,343,344,345,346,347,348,349,350,351,352,353,354,355,356,357,358,359,360,361,362,363,364,365,366,367,368,369,370,371,372,373,374,375,376,377,378,379,380,381,382,383,384,385,386,387,388,389,390,391,392,393,394,395,396,397,398,399,400,401,402,403,404,405,406,407,408,409,410,411,412,413,414,415,416,417,418,419,420,421,422,423,424,425,426,427,428,429,430,431,432,433,434,435,436,437,438,439,440,441,442,443,444,445,446,447,448,449,450,451,452,453,454,455,456,457,458,459,460,461,462,463,464,465,466,467,468,469,470,471,472,473,474,475,476,477,478,479,480,481,482,483,484,485,486,487,488,489,490,491,492,493,494,495,496,497,498,499
0,1,-0.083034,0.248762,0.228279,-0.076599,-0.061450,-0.573516,-0.064687,0.338189,-0.040457,-0.126626,-0.073015,-0.422403,-0.061389,-0.078933,0.303551,-0.104491,0.075580,-0.594623,0.137090,-0.456862,0.273341,0.208257,0.335345,-0.043741,-0.103085,0.228161,-0.335923,-0.070504,-0.191927,0.067585,0.080337,0.174546,0.098350,-0.053253,-0.146290,0.377254,-0.115898,-0.337526,-0.298935,-0.480983,-0.034384,-0.160547,-0.141580,0.042635,0.106486,-0.247295,-0.293057,-0.012838,0.129754,0.184983,0.081565,-0.161665,-0.066976,-0.133113,-0.134422,0.149447,0.030397,0.076298,-0.042125,0.004966,0.004264,0.098660,-0.174964,-0.098476,-0.194665,0.189104,-0.074882,0.094425,-0.285810,0.355592,-0.408823,0.128934,0.331584,-0.098682,0.258098,0.084635,0.031238,-0.121081,-0.258093,0.118332,-0.175984,0.096421,-0.194163,0.474888,0.038900,-0.072510,-0.030213,0.344505,0.484820,0.059054,0.434348,-0.003078,0.130097,0.023512,0.377790,0.064665,0.175259,-0.197856,0.122798,0.042811,-0.024674,0.169980,0.144459,0.001760,-0.027566,-0.292535,0.001164,0.218504,-0.046959,-0.081415,-0.038941,-0.243365,-0.009410,0.009599,0.109168,-0.098398,0.070268,-0.299186,0.096933,-0.217602,0.153524,0.125258,0.132779,-0.045518,-0.062042,0.104743,-0.212267,-0.053418,-0.111603,0.022658,0.056596,0.080235,-0.000666,-0.071295,-0.045399,0.222930,-0.028088,-0.151354,-0.142275,-0.259564,-0.003327,-0.097722,-0.023727,0.008074,0.063804,-0.101525,-0.149739,-0.005279,0.042578,0.113967,0.060115,-0.086169,-0.047568,-0.066455,-0.089247,0.087067,0.060934,0.047086,-0.077089,-0.008756,-0.015820,0.061702,-0.075481,-0.011290,-0.111760,0.099218,-0.033265,0.012924,-0.173100,0.155532,-0.182099,0.077658,0.184797,-0.095787,0.161637,0.056881,0.008078,-0.039705,-0.153420,0.062223,-0.116920,0.040170,-0.083027,0.255017,-0.002143,-0.013562,-0.003385,0.204902,0.195282,0.059647,0.185001,-0.032214,0.044055,0.003950,0.187216,0.020127,0.056300,-0.109957,0.034342,0.044290,-0.006796,0.085749,0.069529,0.001705,-0.019460,-0.139867,0.001358,0.102558,-0.021944,-0.039341,-0.018006,-0.115549,-0.005623,0.003379,0.053114,-0.044986,0.039627,-0.132736,0.047909,-0.105582,0.075881

Теперь у нас для каждого пользователя готов вектор его предпочтений, который можно сравнить с фильмами и порекомендовать наиболее подходящие

**6 Найдем топ близких фильмов, которые пользователь еще не смотрел**

Мы будем использовать косинусное расстояние между вектором предпочтений пользователя и векторами фильмов. Косинусное расстояние – это мера похожести двух векторов. Скалярное произведение векторов и косинус угла θ между ними связаны следующим соотношением: a * b = ||a|| ||b|| cosθ. Имея два вектора A и B, получаем косинусное расстояние – cos(θ)

In [56]:
def get_closest_films(usr_avg_vec_df, user_id, films_emb_mtrx, user_data_dict, top_n=15):

  '''
     на вход подается:
     - usr_avg_vec_df - матрица с train-предпочтениями пользователя
     - user_id - айди пользователя, для которого мы хотим получить рекомендации
     - films_emb_mtrx - эмбединг матрица фильмов
     - splitted_data_dict - словарь с train_test data для всех пользователей
     - top_n - количество рекомендуемых фильмов

     на выходе:
     - список наиболее подходящих непросмотренных фильмов
  '''

  scores = {}

  # достаем средний вектор предпочтений пользователя
  user_avg_vec = usr_avg_vec_df[usr_avg_vec_df['Id'] == user_id] \
                                .drop('Id', axis=1) \
                                .values \
                                .flatten() \
                                .tolist()

  # итерируемся по списку фильмов и находим косинусное расстояние между
  # средним вектором пользователя и каждым фильмом
  for film in films_emb_mtrx['movieId']:

    film_vec = films_emb_mtrx[films_emb_mtrx['movieId'] == film] \
                              .drop('movieId', axis=1) \
                              .values \
                              .flatten() \
                              .tolist()

    # промежуточные результаты записываем в словарь
    scores[film] = 1 - spatial.distance.cosine(user_avg_vec, film_vec)

  # удаляем из словаря фильмы, которые пользователь уже смотрел (в train-наборе)
  for viewed_film in user_data_dict[user_id][0]:
    del scores[viewed_film]

  # возвращаем топ фильмов в порядке убывания близости к вектору пользователя
  # записываем их в пятый список под айди пользователя в словаре с train-test-data
  user_data_dict[user_id].append(heapq.nlargest(top_n, scores, key=scores.get))

In [57]:
# дополним словарь предсказаниями по каждому пользователю
for i, user in enumerate(splitted_data.keys()):
  print('Running:', i + 1, '/', len(splitted_data))
  get_closest_films(res_vec_df, user, matrix_emb, splitted_data)

Running: 1 / 544


/usr/local/lib/python3.7/dist-packages/scipy/spatial/distance.py:720: RuntimeWarning: invalid value encountered in double_scalars
  dist = 1.0 - uv / np.sqrt(uu * vv)


Running: 2 / 544
Running: 3 / 544
Running: 4 / 544
Running: 5 / 544
Running: 6 / 544
Running: 7 / 544
Running: 8 / 544
Running: 9 / 544
Running: 10 / 544
Running: 11 / 544
Running: 12 / 544
Running: 13 / 544
Running: 14 / 544
Running: 15 / 544
Running: 16 / 544
Running: 17 / 544
Running: 18 / 544
Running: 19 / 544
Running: 20 / 544
Running: 21 / 544
Running: 22 / 544
Running: 23 / 544
Running: 24 / 544
Running: 25 / 544
Running: 26 / 544
Running: 27 / 544
Running: 28 / 544
Running: 29 / 544
Running: 30 / 544
Running: 31 / 544
Running: 32 / 544
Running: 33 / 544
Running: 34 / 544
Running: 35 / 544
Running: 36 / 544
Running: 37 / 544
Running: 38 / 544
Running: 39 / 544
Running: 40 / 544
Running: 41 / 544
Running: 42 / 544
Running: 43 / 544
Running: 44 / 544
Running: 45 / 544
Running: 46 / 544
Running: 47 / 544
Running: 48 / 544
Running: 49 / 544
Running: 50 / 544
Running: 51 / 544
Running: 52 / 544
Running: 53 / 544
Running: 54 / 544
Running: 55 / 544
Running: 56 / 544
Running: 57 / 544


Мы получили топ n предсказаний для каждого пользователя, теперь можно переходить к оценки результатов по основным метрикам

**7 Напишем функции для метрик и оцениваем алгоритм**

Precision можно интепретировать как долю релевантных рекомендаций, мы дополнительно считаем среднее по всем пользователям

Precision = TP / (TP + FP), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FP - это все рекомендованные объекты, то есть K - количество предсказанных объектов

In [58]:
def mean_precision(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика precision@k по всем пользователям
  '''

  sum_precision = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    k = len(user_data_dict[user][4])

    sum_precision += TP / k

  # находим среднее по пользователям
  return sum_precision / len(user_data_dict)

Recall интерпретируется как доля релевантных объектов, попавших в рекомендации, мы дополнительно считаем среднее по всем пользователям

Recall = TP / (TP + FN), где TP - фильмы из тестовой выборки, совпавшие с рекомендациями, а TP + FN - это все объекты из тестовой выборки

In [59]:
def mean_recall(user_data_dict):

  '''
     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     средняя метрика recall@k по всем пользователям
  '''

  sum_recall = 0

  # вычисляем метрику по каждому пользователю
  for user in user_data_dict.keys():
    TP = len(set(user_data_dict[user][4]).intersection(set(user_data_dict[user][2])))
    TP_FN = len(user_data_dict[user][2])
    
    sum_recall += TP / TP_FN

  # находим среднее по пользователям
  return sum_recall / len(user_data_dict)

Mean Reciprocal Rank - средний обратный ранк первого правильного вхождения рекомендации в тестовые данные

In [60]:
def MRR(user_data_dict):

  '''
     RR - обратный ранг, 1 / индеск первого вхождения рекомендации в тест

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MRR@k по всем пользователям
  '''

  sum_rr = 0

  # находим порядковый номер первого вхождения правильной рекомендации в тест
  for user in user_data_dict.keys():
    rank = [1 + splitted_data[user][4].index(x) for x in \
            set(splitted_data[user][4]).intersection(set(splitted_data[user][2]))]
    
    # находим обратный ранк, если попадания нет, то зануляем
    if len(rank) == 0:
      rr = 0
    else:
      rr = 1 / min(rank)

    sum_rr += rr

  # находим среднее по пользователям
  return sum_rr / len(user_data_dict)

Average Precision дополнительно вознаграждает нас за предварительну загрузку рекомендаций, которые будут правильными, то есть здесь, в отличие от Precision, учитывается порядок выдаваемых рекомендаций и их точность отдельно. MAP - это среднее AP по всем пользователям

In [61]:
def mean_average_precision_at_k(user_data_dict):

  '''
     Mean Average Precision - средняя точность по пользователям

     на вход подается:
     - user_data_dict - cловарь с данными о пользователях
     (нас интересует список с индексом 2 - test фильмы и с индексом 4 - рекомендации)

     k берется таким, для которого мы рассчитывали рекомендации (длина рекомендаций)

     на выходе:
     метрика MAP@k по всем пользователям
  '''

  sum_ap = 0

  # итерируемся по пользователям
  for user in user_data_dict.keys():

    num_hits = 0
    score = 0

    for i, p in enumerate(user_data_dict[user][4]):

      if p in user_data_dict[user][2] and p not in user_data_dict[user][4][:i]:
        num_hits += 1
        score += num_hits / (i + 1)

    sum_ap += score / min(len(user_data_dict[user][2]), len(user_data_dict[user][4]))

  # находим среднее по пользователям
  return sum_ap / len(user_data_dict)

**Результат:**

In [62]:
print('Mean Precision: ', round(mean_precision(splitted_data), 6))
print('Mean Recall: ', round(mean_recall(splitted_data), 6))
print('Mean Reciprocal Rank:', round(MRR(splitted_data), 6))
print('Mean Average Precision at K:', round(mean_average_precision_at_k(splitted_data), 6))

Mean Precision:  0.003186
Mean Recall:  0.003111
Mean Reciprocal Rank: 0.009492
Mean Average Precision at K: 0.00076
